<a href="https://colab.research.google.com/github/bugrakaann/SuspiciousActionRecognition/blob/master/x3dxs_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorchvideo torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:

In [2]:
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
from google.colab import drive
import zipfile

drive.mount('/content/drive')

!unzip "/content/drive/MyDrive/dataset.zip" -d "/content/sample_data"

Mounted at /content/drive
Archive:  /content/drive/MyDrive/dataset.zip
   creating: /content/sample_data/dataset/
   creating: /content/sample_data/dataset/Test_Train_Splits/
   creating: /content/sample_data/dataset/Test_Train_Splits/50%/
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split1.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split2.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split3.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split4.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/fall_test_split5.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/grab_test_split1.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/grab_test_split2.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/grab_test_split3.txt  
  inflating: /content/sample_data/dataset/Test_Train_Splits/50%/

In [4]:
import os
import pandas as pd

# 🔥 Ana dataset klasörü
dataset_path = "/content/sample_data/dataset/Videos/Videos/"

# 📌 Normal ve Abnormal klasörlerini al
normal_path = os.path.normpath(os.path.join(dataset_path, "normal"))
abnormal_path = os.path.normpath(os.path.join(dataset_path, "abnormal"))

# 📌 Tüm abnormal sınıfları al
abnormal_classes = [folder for folder in os.listdir(abnormal_path) if os.path.isdir(os.path.join(abnormal_path, folder))]
class_to_index = {cls: idx for idx, cls in enumerate(abnormal_classes, start=1)}  # 1, 2, 3... (Abnormal sınıflar)

# 📌 DataFrame oluşturmak için liste
data = []

# ✅ Normal videolar (Binary = 0, Multi = -1)
for subfolder in os.listdir(normal_path):
    subfolder_path = os.path.normpath(os.path.join(normal_path, subfolder))

    if os.path.isdir(subfolder_path):  # Eğer bir klasörse
        videos = [
            os.path.normpath(os.path.join(subfolder_path, v)).replace("\\", "/")  # Windows uyumu için düzeltilmiş
            for v in os.listdir(subfolder_path) if v.endswith(('.mp4', '.avi', '.mov'))
        ]
        for video in videos:
            data.append((video, "normal", 0, -1))  # Binary label = 0, Multi label = -1 (yok)

# ✅ Anormal videolar (Binary = 1, Multi = class index)
for subfolder in abnormal_classes:
    subfolder_path = os.path.normpath(os.path.join(abnormal_path, subfolder))

    if os.path.isdir(subfolder_path):
        # Rekürsif olarak tüm alt klasörleri gezer
        for root, dirs, files in os.walk(subfolder_path):
            for file in files:
                if file.endswith(('.mp4', '.avi', '.mov')):
                    video_path = os.path.normpath(os.path.join(root, file)).replace("\\", "/")
                    # Burada 'subfolder' en üst düzey klasör adını temsil eder;
                    # eğer alt klasörlere göre farklı etiket atamak isterseniz, root veya dosya adını parse edebilirsiniz.
                    data.append((video_path, subfolder, 1, class_to_index.get(subfolder, -1)))

# 🔥 DataFrame oluştur
df = pd.DataFrame(data, columns=["video_path", "category", "binary_label", "multi_label"])


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2,stratify=df["binary_label"],random_state=42)

print(f"Train set {len(train_df)} videos")
print(f"Test set {len(test_df)} videos")

Train set 2603 videos
Test set 651 videos


In [ ]:
# 📌 Google Colab için başlangıç: RAFT deposunu klonla
!git clone https://github.com/princeton-vl/RAFT.git
!mkdir -p RAFT/models

# ✅ Google Drive'dan raft-things.pth dosyasını indir
!cp "/content/drive/MyDrive/raft-things.pth" "/content/RAFT/models/raft-things.pth"

import sys
sys.path.append('/content/RAFT/core')

import os
import gc
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import pandas as pd
from tqdm import tqdm
from raft import RAFT
from argparse import Namespace

# 📌 Bellek Yönetimi: GPU belleğini temizle
torch.cuda.empty_cache()
gc.collect()

# 📌 CUDA ayarları
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# 📌 RAFT modelini yükle (Things modeli)
args = Namespace(small=False, mixed_precision=True, alternate_corr=False)
raft_model = RAFT(args).to(device)
raft_model.load_state_dict(torch.load("/content/RAFT/models/raft-things.pth", map_location=device), strict=False)
raft_model.eval()

print(f"✅ RAFT Model {device}'a yüklendi (RAFT Things)")

def extract_frames(video_path, stride=4):
    """Videodan belirli aralıklarda frame çıkar.
    Frame’ler CPU’da tutuluyor, böylece GPU belleği verimli kullanılıyor."""
    vr = torchvision.io.VideoReader(video_path, "video")
    vr.set_current_stream("video")

    frames = []
    for i, frame in enumerate(vr):
        if i % stride == 0:
            frame = frame['data'].float() / 255.0
            frame = TF.resize(frame, (192, 192))
            # GPU’ya taşımıyoruz, CPU'da saklıyoruz
            frames.append(frame)
    return frames

def compute_optical_flow_raft(video_path, num_segments=5, segment_length=64, stride=4, initial_batch_size=16):
    """RAFT Optical Flow hesaplama (Memory Management + Dynamic Batch).
    Frame’ler CPU’da işlenip, her batch GPU’ya taşınıyor."""
    frames = extract_frames(video_path, stride=stride)
    movement_scores = []

    if len(frames) < 2:
        return []

    # Frame çiftlerini CPU’da oluşturuyoruz
    frame_pairs = torch.stack([
        torch.cat([frames[i].unsqueeze(0), frames[i+1].unsqueeze(0)], dim=0)
        for i in range(len(frames)-1)
    ])

    batch_size = initial_batch_size

    with torch.no_grad():
        for i in range(0, len(frame_pairs), batch_size):
            while batch_size > 0:
                # İşleme başlamadan batch’ı GPU’ya taşıyoruz
                batch = frame_pairs[i:i+batch_size].to(device)
                if batch.shape[0] == 0:
                    break

                try:
                    with torch.cuda.amp.autocast():
                        flow = raft_model(batch[:, 0], batch[:, 1])[0]

                    batch_movement = torch.sum(torch.abs(flow), dim=[1, 2, 3])
                    movement_scores.extend(batch_movement.cpu().numpy())

                    # Bellek temizliği
                    del batch, flow
                    torch.cuda.empty_cache()
                    gc.collect()

                    break  # İşlem başarılı, iç döngüden çık

                except torch.cuda.OutOfMemoryError:
                    # OOM hatası durumunda batch size’ı düşür
                    torch.cuda.empty_cache()
                    gc.collect()
                    batch_size = max(1, batch_size // 2)
                    print(f"⚠️ OOM Hatası! Yeni batch size: {batch_size}")

                    if batch_size == 1:
                        print("🚨 Uyarı: En düşük batch size = 1 olmasına rağmen OOM hatası devam ediyor. İşlem atlanıyor.")
                        return []

    sorted_indices = torch.tensor(movement_scores).argsort(descending=True)[:num_segments]
    return sorted_indices.cpu().tolist()

# 📌 Tüm dataset için hesapla ve CSV'ye kaydet
dataset_folder = "/content/sample_data/dataset/Videos/Videos/abnormal"
output_csv = "/content/drive/MyDrive/modelepochsaves/optical_flow_segments.csv"

video_files = []
for root, dirs, files in os.walk(dataset_folder):
    for file in files:
        if file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.normpath(os.path.join(root, file)).replace("\\", "/")
            video_files.append(video_path)

# 🚀 İlerleme çubuğu ile video işleme
data = []
for video_path in tqdm(video_files, desc="Processing Videos", unit="video"):
    segments = compute_optical_flow_raft(video_path, stride=4, initial_batch_size=8)
    data.append({"video_path": video_path, "top_segments": segments})

df = pd.DataFrame(data)
df.to_csv(output_csv, index=False)

print(f"✅ RAFT Optical Flow hesaplaması tamamlandı ve {output_csv} dosyasına kaydedildi!")

Cloning into 'RAFT'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 147 (delta 0), reused 0 (delta 0), pack-reused 146 (from 2)
Receiving objects: 100% (147/147), 10.01 MiB | 43.64 MiB/s, done.
Resolving deltas: 100% (57/57), done.
✅ RAFT Model cuda'a yüklendi (RAFT Things)


Processing Videos:   0%|          | 0/2408 [00:00<?, ?video/s]<ipython-input-6-b88d4507ebc5>:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/content/RAFT/core/raft.py:99: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):
/content/RAFT/core/raft.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/RAFT/c

In [6]:
#Augmentation

import torchvision.transforms as T

frame_transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=(-10, 10)),
    T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    T.RandomApply([T.GaussianBlur(3)], p=0.3),
    T.RandomApply([T.RandomGrayscale(p=0.2)], p=0.2),
])



In [7]:
import torch

def frame_dropout(frames, p=0.2):
    """Belirli bir ihtimalle rastgele kareleri düşür."""
    keep_mask = torch.rand(len(frames)) > p
    return frames[keep_mask] if keep_mask.sum() > 0 else frames

def speed_perturbation(frames, min_speed=0.8, max_speed=1.2):
    """Videoyu rastgele hızlandır veya yavaşlat."""
    speed_factor = torch.FloatTensor(1).uniform_(min_speed, max_speed).item()
    indices = torch.linspace(0, len(frames)-1, int(len(frames) * speed_factor)).long()
    return frames[indices]

In [8]:
#Fixes frame count after applying frame dropout at augmentation

def fix_frame_count(frames, target_num_frames):
    """Frame sayısını sabitler: Eğer fazla ise kırpar, eksikse tekrarlar."""
    num_frames = frames.shape[0]  # Frame sayısını al

    if num_frames > target_num_frames:
        frames = frames[:target_num_frames]  # Fazla olanları kırp
    elif num_frames < target_num_frames:
        repeat_frames = target_num_frames - num_frames
        last_frame = frames[-1].unsqueeze(0).repeat(repeat_frames, 1, 1, 1)  # Eksik frame'leri tekrar et
        frames = torch.cat([frames, last_frame], dim=0)

    return frames  # Her durumda (T, C, H, W) döner

In [9]:
import torch
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
import random

class VideoDataset(Dataset):
    def __init__(self, df, optical_flow_csv, num_frames=8, transform=None, resize_size=(192, 192), mode= "train"):
        self.df = df
        self.optical_flow_data = pd.read_csv(optical_flow_csv)  # 📌 Optical Flow verileri
        self.num_frames = num_frames
        self.transform = transform
        self.resize_size = resize_size
        self.mode = mode

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        video_path = self.df.iloc[idx]["video_path"]
        binary_label = self.df.iloc[idx]["binary_label"]
        multi_label = self.df.iloc[idx]["multi_label"]

        if binary_label == 1:  # **Anormal Video**
            # 📌 **Önceden hesaplanan en hareketli segmentleri al**
            segments = eval(self.optical_flow_data[self.optical_flow_data["video_path"] == video_path]["top_segments"].values[0])

            # 📌 **Rastgele bir hareketli segment seç**
            chosen_segment = random.choice(segments)

            # 📌 **Seçili segmentin frame'lerini al**
            frames = self.extract_frames(video_path, chosen_segment)

        else:  # **Normal Video**
            frames = self.extract_uniform_frames(video_path, self.num_frames)

        frames = fix_frame_count(frames, self.num_frames)

        #Eğer train setindeysek augmentation uygula
        if self.mode == "train":
            frames = frame_dropout(frames, p=0.1)
            frames = speed_perturbation(frames, min_speed=0.8, max_speed=1.2)
            frames = torch.stack([frame_transform(frame) for frame in frames])
            frames = fix_frame_count(frames,self.num_frames)


        # **🎯 GİRİŞ FORMATINI DÜZELTME**
        frames = frames.permute(1, 0, 2, 3).contiguous()  # (T, C, H, W) → (C, T, H, W)

        # Eğer transform varsa uygula
        if self.transform:
            frames = self.transform(frames)

        return frames, torch.tensor(binary_label, dtype=torch.float32), torch.tensor(multi_label, dtype=torch.long)

    def extract_frames(self, video_path, segment_index):
        """Anormal videolar için Optical Flow'a göre seçilen segmentten frame'leri çıkarır."""
        vr = torchvision.io.VideoReader(video_path, "video")
        vr.set_current_stream("video")

        frames = []
        frame_count = 0
        start_frame = segment_index  # 📌 Seçilen hareketli segmentin başlangıcı
        end_frame = start_frame + self.num_frames

        for i, frame in enumerate(vr):
            if i < start_frame:
                continue  # 📌 Seçilen segmentin başlangıcına kadar frame'leri atla

            if i >= end_frame:
                break  # 📌 Seçilen segment tamamlandıktan sonra çık

            frame = frame['data'].float() / 255.0  # Normalize (C, H, W)
            frame = F.interpolate(frame.unsqueeze(0), size=self.resize_size, mode="bilinear", align_corners=False)
            frames.append(frame.squeeze(0))

            frame_count += 1

        return torch.stack(frames)  # (T, C, H, W) olarak döndür

    def extract_uniform_frames(self, video_path, num_frames):
        """Normal videolar için eşit aralıklarla frame seçer."""
        vr = torchvision.io.VideoReader(video_path, "video")
        vr.set_current_stream("video")

        frames = []
        for frame in vr:
            frame = frame['data'].float() / 255.0  # Normalize (C, H, W)
            frame = F.interpolate(frame.unsqueeze(0), size=self.resize_size, mode="bilinear", align_corners=False)
            frames.append(frame.squeeze(0))

        total_frames = len(frames)

        # Eğer video kısa ise, son frame ile doldur
        if total_frames < num_frames:
            while len(frames) < num_frames:
                frames.append(frames[-1].clone())
        else:
            # 📌 **Frame'leri eşit aralıklarla seç**
            index_list = torch.linspace(0, total_frames - 1, num_frames).long()
            frames = [frames[i] for i in index_list]

        return torch.stack(frames)  # (T, C, H, W)

In [10]:
train_dataset = VideoDataset(train_df, "/content/drive/MyDrive/modelepochsaves/optical_flow_segments.csv", num_frames=48, mode="train")
test_dataset = VideoDataset(test_df, "/content/drive/MyDrive/modelepochsaves/optical_flow_segments.csv", num_frames=48, mode="test")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"Train loader {len(train_loader)} batches")
print(f"Test loader {len(test_loader)} batches")

Train loader 326 batches
Test loader 82 batches


X3D modelimizin output layerini tüm classlarımızı kapsayacak şekilde arttırmamız lazım
Son katman ise ResNetBasicHead altında proj olarak tanımlanmış.

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

# ✅ X3D Modelini Yükleme
model_name = "x3d_s"
base_model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)

# ✅ **Çıkış Katmanlarını Güncelle**
in_features = base_model.blocks[-1].proj.out_features  # ✅ **Doğru çıkışı al!** (400 olmalı)

binary_head = nn.Linear(2048, 1)  # Binary sınıflandırma için Sigmoid
multi_head = nn.Linear(2048, len(train_df["multi_label"].unique()))  # Multi-class sınıflandırma

# ✅ **MultiTaskX3D Modeli**
class MultiTaskX3D(nn.Module):
    def __init__(self, base_model, binary_head, multi_head):
        super(MultiTaskX3D, self).__init__()

        # **Feature Extractor - Son bloğun içindeki ProjectedPool eklendi!**
        self.feature_extractor = nn.Sequential(
            *base_model.blocks[:-1],
            base_model.blocks[-1].pool  # ✅ **Burada 400'e dönüşüm sağlandı!**
        )

        # **Global Average Pooling - Tüm boyutları 1x1x1'e düşürüyor!**
        self.gap = nn.AdaptiveAvgPool3d(1)

        # **Flatten**
        self.flatten = nn.Flatten()

        # **Çıkış Katmanları**
        self.binary_head = binary_head
        self.binary_activation = nn.Sigmoid()

        self.multi_head = multi_head
        self.multi_activation = nn.Softmax(dim=1)

    def forward(self, x):
        #print(f"🚀 Model Girişi: {x.shape}")

        x = self.feature_extractor(x)
        #print(f"🟡 Feature Extractor Çıkışı: {x.shape}")  # **(16, 400, 1, 1, 1) olmalı!**

        x = self.gap(x)  # ✅ **GAP ile (16, 400, 1, 1, 1)**
        #print(f"🔵 GAP Sonrası Boyut: {x.shape}")

        x = self.flatten(x)  # ✅ **Artık (16, 400) olması lazım!**
        #print(f"🟢 Flatten Sonrası Boyut: {x.shape}")

        binary_out = self.binary_head(x)  # Binary sınıflandırma
        multi_out = self.multi_activation(self.multi_head(x))  # Multi-class sınıflandırma

        return binary_out, multi_out

# ✅ Modeli başlat
model = MultiTaskX3D(base_model, binary_head, multi_head)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print(f"🚀 Model başarıyla yüklendi! Çalıştırılan cihaz: {device}")

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/X3D_S.pyth" to /root/.cache/torch/hub/checkpoints/X3D_S.pyth
100%|██████████| 29.4M/29.4M [00:00<00:00, 38.7MB/s]


🚀 Model başarıyla yüklendi! Çalıştırılan cihaz: cuda


import torch

model = model.to("cpu")
dummy_input = torch.randn(1, 3, 16, 160, 160)

torch.onnx.export(model, dummy_input, "x3d_xs.onnx", opset_version=11
                  ,do_constant_folding=True, input_names=["input"], output_names=["output"])

In [12]:
from torch.nn import CrossEntropyLoss

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self,inputs,targets):
        ce_loss = F.cross_entropy(inputs,targets,reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        return focal_loss.mean()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

epoch_file_location = "/content/drive/MyDrive/modelepochsaves"

# 🎯 Loss & Optimizer
criterion_binary = nn.BCEWithLogitsLoss()  # Binary classification için
criterion_multi = FocalLoss()  # Multi-class classification için

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-2)  # Weight decay ile overfitting azaltılır
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# ⚡ AMP (Automatic Mixed Precision)
scaler = torch.amp.GradScaler()

# 🛑 Early Stopping Parametreleri
patience = 3  # Kaç epoch boyunca iyileşme olmazsa durdursun
min_delta = 0.01  # İyileşme için gereken minimum fark
best_loss = float("inf")  # En iyi validation loss başta sonsuz olarak ayarlanır
counter = 0  # İyileşme olmayan epoch sayısı
accuracy_threshold = 85.0  # Accuracy'nin geçmesi gereken eşik

device = "cuda" if torch.cuda.is_available() else "cpu"

for epoch in range(40):
    model.train()
    running_loss = 0.0

    for videos, binary_labels, multi_labels in train_loader:
        videos, binary_labels, multi_labels = (
            videos.to(device),
            binary_labels.to(device).float(),
            multi_labels.to(device),
        )

        optimizer.zero_grad()

        with torch.amp.autocast(device):
            binary_out, multi_out = model(videos)

            # 🟢 Binary Classification Loss (Sigmoid)
            binary_loss = criterion_binary(binary_out.squeeze(), binary_labels)

            # 🔴 Multi-Class Classification Loss (Softmax) → Sadece anormal olanlara uygula
            mask = (binary_labels == 1)
            if mask.sum() > 0:
                multi_loss = criterion_multi(multi_out[mask], multi_labels[mask])
            else:
                multi_loss = torch.tensor(0.0, device=device)

            # ✅ Toplam loss hesapla (Binary + Multi)
            total_loss = binary_loss + multi_loss

        scaler.scale(total_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += total_loss.item()

    train_loss = running_loss / len(train_loader)

    # 📌 VALIDATION AŞAMASI
    model.eval()
    val_loss = 0.0
    correct_binary = 0
    correct_multi = 0
    total_binary = 0
    total_multi = 0

    with torch.no_grad():
        for videos, binary_labels, multi_labels in test_loader:
            videos, binary_labels, multi_labels = (
                videos.to(device),
                binary_labels.to(device).float(),
                multi_labels.to(device),
            )

            binary_out, multi_out = model(videos)

            # ✅ Binary Accuracy (Normal mi, Anormal mi?)
            pred_binary = (binary_out.squeeze() > 0.5).long()
            correct_binary += (pred_binary == binary_labels).sum().item()
            total_binary += binary_labels.size(0)

            # ✅ Multi-Class Accuracy (Eğer anormalse hangi sınıf?)
            mask = (binary_labels == 1)
            if mask.sum() > 0:
                pred_multi = torch.argmax(multi_out[mask], dim=1)
                correct_multi += (pred_multi == multi_labels[mask]).sum().item()
                total_multi += multi_labels[mask].size(0)

            # ✅ Validation loss hesapla
            loss = criterion_binary(binary_out.squeeze(), binary_labels)
            if total_multi > 0:
                loss += criterion_multi(multi_out[mask], multi_labels[mask])
            val_loss += loss.item()

    val_loss /= len(test_loader)
    binary_acc = 100 * correct_binary / total_binary
    multi_acc = 100 * correct_multi / total_multi if total_multi > 0 else 0

    print(
        f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} "
        f"| Binary Acc: {binary_acc:.2f}% | Multi Acc: {multi_acc:.2f}%"
    )

    # ✅ **Early Stopping (Binary Accuracy 85%'i geçmeli)**
    if binary_acc >= accuracy_threshold:
        if best_loss - val_loss > min_delta:
            best_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), f"{epoch_file_location}/best_model_epoch_{epoch+1}.pth")
            print(f"✅ Model kaydedildi: best_model_epoch_{epoch+1}.pth")
        else:
            counter += 1
            print(f"❗ Early stopping counter: {counter}/{patience}")

        if counter >= patience:
            print("⏹️ Early stopping triggered. Training stopped.")
            break
    else:
        print(f"🚨 Accuracy {binary_acc:.2f}% < {accuracy_threshold}%, early stopping devre dışı!")

    scheduler.step()

    if (epoch + 1) % 5 == 0:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            },
            f"{epoch_file_location}/checkpoint_{epoch}.pth",
        )
        print(f"Model {epoch + 1} saved to checkpoint_{epoch + 1}.pth")

print("✅ Eğitim tamamlandı!")

Epoch 1 | Train Loss: 0.7108 | Val Loss: 0.7181 | Binary Acc: 60.06% | Multi Acc: 48.96%
🚨 Accuracy 60.06% < 85.0%, early stopping devre dışı!
Epoch 2 | Train Loss: 0.6038 | Val Loss: 0.7167 | Binary Acc: 56.07% | Multi Acc: 53.11%
🚨 Accuracy 56.07% < 85.0%, early stopping devre dışı!
Epoch 3 | Train Loss: 0.5089 | Val Loss: 0.7046 | Binary Acc: 54.99% | Multi Acc: 54.77%
🚨 Accuracy 54.99% < 85.0%, early stopping devre dışı!
